In [1]:
%pylab inline
%matplotlib inline

import sys
import os
import time
import codecs
import pandas as pd
from pprint import pprint
import re

sys.path.append("/Users/Bya/git/predictEPL/utils/")
sys.path.append("/Users/Bya/git/predictEPL/config/")

import emolex
import paths
import tokenizer
import useful_methods as my_methods
import train_datas

Populating the interactive namespace from numpy and matplotlib


In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model.logistic import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV

### Problem: Movie Review 5 Sentiment

In [3]:
###############################

# 1. Data Load

dic_sentiment = {
    "negative": 0,
    "somewhat negative": 1,
    "neutral": 2,
    "somewhat positive": 3,
    "positive": 4
}

os.chdir("/Users/Bya/Dropbox/Research/resources/Kaggle-MovieReviews(RottenTomatos)/")
df = pd.read_csv('train.tsv', header=0, delimiter='\t')
df.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [4]:
# Sentiments
print(df["Sentiment"].value_counts())
print(df["Sentiment"].value_counts() / df['Sentiment'].count())

2    79582
3    32927
1    27273
4     9206
0     7072
Name: Sentiment, dtype: int64
2    0.509945
3    0.210989
1    0.174760
4    0.058990
0    0.045316
Name: Sentiment, dtype: float64


In [7]:
s = df['Sentiment'].as_matrix()
type(s[0])

numpy.int64

In [19]:
####################################

# 2. Train Classifier

# Data Split: Train, Test
X, y = df['Phrase'], df['Sentiment'].as_matrix()

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.5)


# Pipelines
pipeline = Pipeline([
        ('vect', TfidfVectorizer(stop_words='english')),
        ('clf', LogisticRegression())
    ])


# Hyper Parameters
parameters = {
    'vect__max_df': (0.25, 0.5),
    'vect__ngram_range': ((1, 1), (1, 2)),
    'vect__use_idf': (True, False),
    'clf__C': (0.1, 1, 10),
}


# Grid Search
grid_search = GridSearchCV(
    pipeline,
    parameters,
    n_jobs=-1,
    verbose=1,
    scoring='accuracy',
    cv=18
)

In [20]:
# Computing
grid_search.fit(X_train, y_train)

print('Best score: %0.3f' % grid_search.best_score_)
print('Best parameters set:')

best_parameters = grid_search.best_estimator_.get_params()

for param_name in sorted(parameters.keys()):
    print('\t%s: %r' % (param_name, best_parameters[param_name]))


Fitting 3 folds for each of 24 candidates, totalling 72 fits


[Parallel(n_jobs=3)]: Done   1 jobs       | elapsed:    3.0s
[Parallel(n_jobs=3)]: Done  50 jobs       | elapsed:  1.5min
[Parallel(n_jobs=3)]: Done  68 out of  72 | elapsed:  2.5min remaining:    8.8s
[Parallel(n_jobs=3)]: Done  72 out of  72 | elapsed:  2.7min finished


Best score: 0.618
Best parameters set:
	clf__C: 10
	vect__max_df: 0.25
	vect__ngram_range: (1, 2)
	vect__use_idf: False


## Multi-class classification performance metrics

In [24]:
predictions = grid_search.predict(X_test)
print('\n\nAccuracy:', accuracy_score(y_test, predictions))
print('\n\nConfusion Matrix:\n', confusion_matrix(y_test, predictions))
print('\n\nClassification Report:\n', classification_report(y_test, predictions))



Accuracy: 0.636844803281


Confusion Matrix:
 [[ 1129  1689   639    63     8]
 [  903  6111  6064   590    34]
 [  217  3259 32673  3510   166]
 [   21   384  6614  8112  1275]
 [    4    36   481  2380  1668]]


Classification Report:
              precision    recall  f1-score   support

          0       0.50      0.32      0.39      3528
          1       0.53      0.45      0.49     13702
          2       0.70      0.82      0.76     39825
          3       0.55      0.49      0.52     16406
          4       0.53      0.37      0.43      4569

avg / total       0.62      0.64      0.62     78030

